## Exercício 1

Para este exercício, deveremos desenvolver uma classe que implementasse o algoritmo KEM-ElGamal utilizando SageMath.

O algoritmo El Gamal é um algoritmo de cifra assimétrica, que consiste na adaptação do protocolo Diffie-Hellman (DH) num KEM ("Key Encapsulation Mechanism").



As técnicas da família Diffie-Hellman usam as propriedades de um grupo cíclico multiplicativo $$\,\mathbb{Z}_p^\ast\,$$ , no qual *p* é um primo grande. *p* será tal que $$\,\phi(p)\,$$ tem um divisor primo *q* grande.


Neste caso, o grupo multiplicativo $$\,\mathbb{Z}_p^\ast\,$$ de um anel inteiro $$\,\mathbb{Z}_p\,$$

A ordem deste grupo ciclico é p-1. O gerador é um primo cuja ordem é p-1. Assim, precisamos de obter um número primo p, cujo tamanho seja o parametro de segurança.

In [1]:
from binascii import hexlify, unhexlify


class ElGamal:  
    
    def __init__(self):
        self.private_key = 0
        self.public_key = 0
        self.p = 0
        self.g = 0
        self.q = 0
    
    # Esta função deve inicializar a instância, recebendo parâmetro de segurança
    # O parâmetro de segurança é o tamanho em bits da ordem do grupo ciclico
    # Deve gerar as chaves pública e privada
    
    # KeyGen_alpha -> (pk, sk)
    
    def key_gen(self, parametro_seguranca):
        
        self.q = random_prime(2^parametro_seguranca, False, 2^(parametro_seguranca-1))
        
        i = 0
        while not is_prime(self.p):
            self.p = self.q * pow(2, i) + 1
            i = i + 1
        
        # self.p = random_prime(2^parametro_seguranca, False, 2^(parametro_seguranca-1))
        print(self.p)
        print(is_prime(self.p))
        
        print(self.q)
        print(is_prime(self.q))
        
        pI = Integers(self.p) # Ring of integers modulo p
        Zp = pI.unit_group()
        gen = pI.multiplicative_generator()
        o = Zp.order()
    
        print(f"grupo multiplicativo {Zp} com gerador {gen} de ordem {o}\n")

        self.g = pI.random_element((2^self.q))
        
        
        self.private_key = ZZ.random_element(self.q)
        
        self.public_key = (self.g^(self.private_key)) % self.p
        # print(self.public_key) 
            
    
    def KEM(self):
        r = ZZ.random_element(self.q)
        key = pow(self.public_key, r) % self.p
        enc = pow(self.g,r) % self.p
        return key, enc
        

    def KRev(self, enc):
        return pow(enc, self.private_key) % self.p
    
    def Xor(self, message, key):
        output = bytes([x^^y for (x,y) in zip(message,key)])
        # for i, j in zip(message, key):
        #     word = i ^ j # Xor needs to be used between chars
        #     output += word # Guardar esta word
        return bytes(output)
    
    # E(x) = (e,k) <- KEM . (e, k xor x)
    def encrypt(self, message):
        (key, enc) = self.KEM()
        
        key_binary = hexlify(str(key).encode('utf-8'))
        enc_binary = hexlify(str(enc).encode('utf-8'))
        message_binary = hexlify(str(message).encode('utf-8'))
        
        ciphertext = self.Xor(message_binary, key_binary)
        
        return enc, ciphertext # key xor message
    
     # D(e,c) = k <- KRev . (k xor c)
    def decrypt(self, enc, ciphertext):
        key = self.KRev(enc)
        k = hexlify(str(key).encode('utf-8'))
        
        plaintext = self.Xor(ciphertext, k)
        plaintext = unhexlify(plaintext.decode('utf-8'))
        return plaintext
    
a = ElGamal()
a.key_gen(10)
print(a.private_key)
print(a.public_key)

enc, ciphertext = a.encrypt("TESTE")
plaintext = a.decrypt(enc, ciphertext)
print(plaintext)


1283
True
641
True
grupo multiplicativo Multiplicative Abelian group isomorphic to C1282 com gerador 2 de ordem 1282

345
396
b'TES'
